#### Copied from transform.py and interpolate_poses.py resp... 
docker run -it --net=host -p 127.0.0.1:8889:8888 -v /home/sourav/workspace/:/workspace/ -v /media/sourav/Default2/Users/n9349995/Desktop/dataset/:/workspace/dataset/ -v /media/sourav/My\ Passport1/:/workspace/dataset2 souravgarg/opuc

### Transform Functions 

In [ ]:
################################################################################
#
# Copyright (c) 2017 University of Oxford
# Authors:
#  Geoff Pascoe (gmp@robots.ox.ac.uk)
#
# This work is licensed under the Creative Commons
# Attribution-NonCommercial-ShareAlike 4.0 International License.
# To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc-sa/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.
#
################################################################################

import numpy as np
import numpy.matlib as matlib
from math import sin, cos, atan2, sqrt

MATRIX_MATCH_TOLERANCE = 1e-4


def build_se3_transform(xyzrpy):
    """Creates an SE3 transform from translation and Euler angles.

    Args:
        xyzrpy (list[float]): translation and Euler angles for transform. Must have six components.

    Returns:
        numpy.matrixlib.defmatrix.matrix: SE3 homogeneous transformation matrix

    Raises:
        ValueError: if `len(xyzrpy) != 6`

    """
    if len(xyzrpy) != 6:
        raise ValueError("Must supply 6 values to build transform")

    se3 = matlib.identity(4)
    se3[0:3, 0:3] = euler_to_so3(xyzrpy[3:6])
    se3[0:3, 3] = np.matrix(xyzrpy[0:3]).transpose()
    return se3


def euler_to_so3(rpy):
    """Converts Euler angles to an SO3 rotation matrix.

    Args:
        rpy (list[float]): Euler angles (in radians). Must have three components.

    Returns:
        numpy.matrixlib.defmatrix.matrix: 3x3 SO3 rotation matrix

    Raises:
        ValueError: if `len(rpy) != 3`.

    """
    if len(rpy) != 3:
        raise ValueError("Euler angles must have three components")

    R_x = np.matrix([[1, 0, 0],
                     [0, cos(rpy[0]), -sin(rpy[0])],
                     [0, sin(rpy[0]), cos(rpy[0])]])
    R_y = np.matrix([[cos(rpy[1]), 0, sin(rpy[1])],
                     [0, 1, 0],
                     [-sin(rpy[1]), 0, cos(rpy[1])]])
    R_z = np.matrix([[cos(rpy[2]), -sin(rpy[2]), 0],
                     [sin(rpy[2]), cos(rpy[2]), 0],
                     [0, 0, 1]])
    R_zyx = R_z * R_y * R_x
    return R_zyx


def so3_to_euler(so3):
    """Converts an SO3 rotation matrix to Euler angles

    Args:
        so3: 3x3 rotation matrix

    Returns:
        numpy.matrixlib.defmatrix.matrix: list of Euler angles (size 3)

    Raises:
        ValueError: if so3 is not 3x3
        ValueError: if a valid Euler parametrisation cannot be found

    """
    if so3.shape != (3, 3):
        raise ValueError("SO3 matrix must be 3x3")
    roll = atan2(so3[2, 1], so3[2, 2])
    yaw = atan2(so3[1, 0], so3[0, 0])
    denom = sqrt(so3[0, 0] ** 2 + so3[1, 0] ** 2)
    pitch_poss = [atan2(-so3[2, 0], denom), atan2(-so3[2, 0], -denom)]

    R = euler_to_so3((roll, pitch_poss[0], yaw))

    if (so3 - R).sum() < MATRIX_MATCH_TOLERANCE:
        return np.matrix([roll, pitch_poss[0], yaw])
    else:
        R = euler_to_so3((roll, pitch_poss[1], yaw))
        if (so3 - R).sum() > MATRIX_MATCH_TOLERANCE:
            raise ValueError("Could not find valid pitch angle")
        return np.matrix([roll, pitch_poss[1], yaw])


def so3_to_quaternion(so3):
    """Converts an SO3 rotation matrix to a quaternion

    Args:
        so3: 3x3 rotation matrix

    Returns:
        numpy.ndarray: quaternion [w, x, y, z]

    Raises:
        ValueError: if so3 is not 3x3
    """
    if so3.shape != (3, 3):
        raise ValueError("SO3 matrix must be 3x3")

    R_xx = so3[0, 0]
    R_xy = so3[0, 1]
    R_xz = so3[0, 2]
    R_yx = so3[1, 0]
    R_yy = so3[1, 1]
    R_yz = so3[1, 2]
    R_zx = so3[2, 0]
    R_zy = so3[2, 1]
    R_zz = so3[2, 2]

    try:
        w = sqrt(so3.trace() + 1) / 2
    except(ValueError):
        # w is non-real
        w = 0

    x = sqrt(1 + R_xx - R_yy - R_zz) / 2
    y = sqrt(1 + R_yy - R_xx - R_zz) / 2
    z = sqrt(1 + R_zz - R_yy - R_xx) / 2

    max_index = max(range(4), key=[w, x, y, z].__getitem__)

    if max_index == 0:
        x = (R_zy - R_yz) / (4 * w)
        y = (R_xz - R_zx) / (4 * w)
        z = (R_yx - R_xy) / (4 * w)
    elif max_index == 1:
        w = (R_zy - R_yz) / (4 * x)
        y = (R_xy + R_yx) / (4 * x)
        z = (R_zx + R_xz) / (4 * x)
    elif max_index == 2:
        w = (R_xz - R_zx) / (4 * y)
        x = (R_xy + R_yx) / (4 * y)
        z = (R_yz + R_zy) / (4 * y)
    elif max_index == 3:
        w = (R_yx - R_xy) / (4 * z)
        x = (R_zx + R_xz) / (4 * z)
        y = (R_yz + R_zy) / (4 * z)

    return np.array([w, x, y, z])


def se3_to_components(se3):
    """Converts an SE3 rotation matrix to linear translation and Euler angles

    Args:
        se3: 4x4 transformation matrix

    Returns:
        numpy.matrixlib.defmatrix.matrix: list of [x, y, z, roll, pitch, yaw]

    Raises:
        ValueError: if se3 is not 4x4
        ValueError: if a valid Euler parametrisation cannot be found

    """
    if se3.shape != (4, 4):
        raise ValueError("SE3 transform must be a 4x4 matrix")
    xyzrpy = np.empty(6)
    xyzrpy[0:3] = se3[0:3, 3].transpose()
    xyzrpy[3:6] = so3_to_euler(se3[0:3, 0:3])
    return xyzrpy


### Interpolate Functions 

In [ ]:
################################################################################
#
# Copyright (c) 2017 University of Oxford
# Authors:
#  Geoff Pascoe (gmp@robots.ox.ac.uk)
#
# This work is licensed under the Creative Commons
# Attribution-NonCommercial-ShareAlike 4.0 International License.
# To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc-sa/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.
#
################################################################################

import bisect
import csv
import numpy as np
import numpy.matlib as ml
from transform import *


def interpolate_vo_poses(vo_path, pose_timestamps, origin_timestamp):
    """Interpolate poses from visual odometry.

    Args:
        vo_path (str): path to file containing relative poses from visual odometry.
        pose_timestamps (list[int]): UNIX timestamps at which interpolated poses are required.
        origin_timestamp (int): UNIX timestamp of origin frame. Poses will be reported relative to this frame.

    Returns:
        list[numpy.matrixlib.defmatrix.matrix]: SE3 matrix representing interpolated pose for each requested timestamp.

    """
    with open(vo_path) as vo_file:
        vo_reader = csv.reader(vo_file)
        headers = next(vo_file)

        vo_timestamps = [0]
        abs_poses = [ml.identity(4)]

        lower_timestamp = min(min(pose_timestamps), origin_timestamp)
        upper_timestamp = max(max(pose_timestamps), origin_timestamp)

        for row in vo_reader:
            timestamp = int(row[0])
            if timestamp < lower_timestamp:
                vo_timestamps[0] = timestamp
                continue

            vo_timestamps.append(timestamp)

            xyzrpy = [float(v) for v in row[2:8]]
            rel_pose = build_se3_transform(xyzrpy)
            abs_pose = abs_poses[-1] * rel_pose
            abs_poses.append(abs_pose)

            if timestamp >= upper_timestamp:
                break

    return interpolate_poses(vo_timestamps, abs_poses, pose_timestamps, origin_timestamp)


def interpolate_ins_poses(ins_path, pose_timestamps, origin_timestamp):
    """Interpolate poses from INS.

    Args:
        ins_path (str): path to file containing poses from INS.
        pose_timestamps (list[int]): UNIX timestamps at which interpolated poses are required.
        origin_timestamp (int): UNIX timestamp of origin frame. Poses will be reported relative to this frame.

    Returns:
        list[numpy.matrixlib.defmatrix.matrix]: SE3 matrix representing interpolated pose for each requested timestamp.

    """
    with open(ins_path) as ins_file:
        ins_reader = csv.reader(ins_file)
        headers = next(ins_file)

        ins_timestamps = [0]
        abs_poses = [ml.identity(4)]

        upper_timestamp = max(max(pose_timestamps), origin_timestamp)

        for row in ins_reader:
            timestamp = int(row[0])
            ins_timestamps.append(timestamp)

            xyzrpy = [float(v) for v in row[2:8]]
            abs_pose = build_se3_transform(xyzrpy)
            abs_poses.append(abs_pose)

            if timestamp >= upper_timestamp:
                break

    ins_timestamps = ins_timestamps[1:]
    abs_poses = abs_poses[1:]

    return interpolate_poses(ins_timestamps, abs_poses, pose_timestamps, origin_timestamp)


def interpolate_poses(pose_timestamps, abs_poses, requested_timestamps, origin_timestamp):
    """Interpolate between absolute poses.

    Args:
        pose_timestamps (list[int]): Timestamps of supplied poses. Must be in ascending order.
        abs_poses (list[numpy.matrixlib.defmatrix.matrix]): SE3 matrices representing poses at the timestamps specified.
        requested_timestamps (list[int]): Timestamps for which interpolated timestamps are required.
        origin_timestamp (int): UNIX timestamp of origin frame. Poses will be reported relative to this frame.

    Returns:
        list[numpy.matrixlib.defmatrix.matrix]: SE3 matrix representing interpolated pose for each requested timestamp.

    Raises:
        ValueError: if pose_timestamps and abs_poses are not the same length
        ValueError: if pose_timestamps is not in ascending order

    """
    requested_timestamps.insert(0, origin_timestamp)
    requested_timestamps = np.array(requested_timestamps)
    pose_timestamps = np.array(pose_timestamps)

    if len(pose_timestamps) != len(abs_poses):
        raise ValueError('Must supply same number of timestamps as poses')

    abs_quaternions = np.zeros((4, len(abs_poses)))
    abs_positions = np.zeros((3, len(abs_poses)))
    for i, pose in enumerate(abs_poses):
        if i > 0 and pose_timestamps[i-1] >= pose_timestamps[i]:
            raise ValueError('Pose timestamps must be in ascending order')

        abs_quaternions[:, i] = so3_to_quaternion(pose[0:3, 0:3])
        abs_positions[:, i] = np.ravel(pose[0:3, 3])

    upper_indices = [bisect.bisect(pose_timestamps, pt) for pt in requested_timestamps]
    lower_indices = [u - 1 for u in upper_indices]

    if max(upper_indices) >= len(pose_timestamps):
        upper_indices = [min(i, len(pose_timestamps) - 1) for i in upper_indices]

    fractions = (requested_timestamps - pose_timestamps[lower_indices]) / \
                (pose_timestamps[upper_indices] - pose_timestamps[lower_indices])

    quaternions_lower = abs_quaternions[:, lower_indices]
    quaternions_upper = abs_quaternions[:, upper_indices]

    d_array = (quaternions_lower * quaternions_upper).sum(0)

    linear_interp_indices = np.nonzero(d_array >= 1)
    sin_interp_indices = np.nonzero(d_array < 1)

    scale0_array = np.zeros(d_array.shape)
    scale1_array = np.zeros(d_array.shape)

    scale0_array[linear_interp_indices] = 1 - fractions[linear_interp_indices]
    scale1_array[linear_interp_indices] = fractions[linear_interp_indices]

    theta_array = np.arccos(np.abs(d_array[sin_interp_indices]))

    scale0_array[sin_interp_indices] = \
        np.sin((1 - fractions[sin_interp_indices]) * theta_array) / np.sin(theta_array)
    scale1_array[sin_interp_indices] = \
        np.sin(fractions[sin_interp_indices] * theta_array) / np.sin(theta_array)

    negative_d_indices = np.nonzero(d_array < 0)
    scale1_array[negative_d_indices] = -scale1_array[negative_d_indices]

    quaternions_interp = np.tile(scale0_array, (4, 1)) * quaternions_lower \
                         + np.tile(scale1_array, (4, 1)) * quaternions_upper

    positions_lower = abs_positions[:, lower_indices]
    positions_upper = abs_positions[:, upper_indices]

    positions_interp = np.multiply(np.tile((1 - fractions), (3, 1)), positions_lower) \
                       + np.multiply(np.tile(fractions, (3, 1)), positions_upper)

    poses_mat = ml.zeros((4, 4 * len(requested_timestamps)))

    poses_mat[0, 0::4] = 1 - 2 * np.square(quaternions_interp[2, :]) - \
                         2 * np.square(quaternions_interp[3, :])
    poses_mat[0, 1::4] = 2 * np.multiply(quaternions_interp[1, :], quaternions_interp[2, :]) - \
                         2 * np.multiply(quaternions_interp[3, :], quaternions_interp[0, :])
    poses_mat[0, 2::4] = 2 * np.multiply(quaternions_interp[1, :], quaternions_interp[3, :]) + \
                         2 * np.multiply(quaternions_interp[2, :], quaternions_interp[0, :])

    poses_mat[1, 0::4] = 2 * np.multiply(quaternions_interp[1, :], quaternions_interp[2, :]) \
                         + 2 * np.multiply(quaternions_interp[3, :], quaternions_interp[0, :])
    poses_mat[1, 1::4] = 1 - 2 * np.square(quaternions_interp[1, :]) \
                         - 2 * np.square(quaternions_interp[3, :])
    poses_mat[1, 2::4] = 2 * np.multiply(quaternions_interp[2, :], quaternions_interp[3, :]) - \
                         2 * np.multiply(quaternions_interp[1, :], quaternions_interp[0, :])

    poses_mat[2, 0::4] = 2 * np.multiply(quaternions_interp[1, :], quaternions_interp[3, :]) - \
                         2 * np.multiply(quaternions_interp[2, :], quaternions_interp[0, :])
    poses_mat[2, 1::4] = 2 * np.multiply(quaternions_interp[2, :], quaternions_interp[3, :]) + \
                         2 * np.multiply(quaternions_interp[1, :], quaternions_interp[0, :])
    poses_mat[2, 2::4] = 1 - 2 * np.square(quaternions_interp[1, :]) - \
                         2 * np.square(quaternions_interp[2, :])

    poses_mat[0:3, 3::4] = positions_interp
    poses_mat[3, 3::4] = 1

    poses_mat = np.linalg.solve(poses_mat[0:4, 0:4], poses_mat)

    poses_out = [0] * (len(requested_timestamps) - 1)
    for i in range(1, len(requested_timestamps)):
        poses_out[i - 1] = poses_mat[0:4, i * 4:(i + 1) * 4]

    return poses_out


## Methods 

### Pose Conversions and Trajectory Plot Comparison 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import interpolate


#### Load some variables 

In [ ]:
folderPath = "/workspace/dataset2/current/data/oxford/"
# dataId = "2014-12-09-13-21-02/"
dataId = "2014-12-10-18-10-50/"

#### Convert relative poses to global and then convert it into xyzrpy components 

In [ ]:
def getInterpVO(voFile1,voData1,dataLim=6000):
    # Get global SE3 poses
    interpPoses = interpolate_vo_poses(voFile1,voData1[:dataLim,0].tolist(),voData1[0,0])
    print(len(interpPoses),interpPoses[0].shape)

    # Convert SE3 to xyzrpy
    compPoses = []
    for i in range(len(interpPoses)):
        compPoses.append(se3_to_components(interpPoses[i]))
    print(len(compPoses),compPoses[0].shape)
    return np.array(compPoses)


In [ ]:
# Load vo
voFile = folderPath+dataId+"vo/vo.csv"
voData = np.loadtxt(voFile,delimiter=",",skiprows=1)
print("VO Data Shape - ",voData.shape)

compPosesArr = getInterpVO(voFile,voData,voData.shape[0])
# Plot the gt vo
print(compPosesArr.shape)
plt.plot(compPosesArr[:,0],compPosesArr[:,1])

In [ ]:
# Plot the calculated vo
vo_calc = np.loadtxt(folderPath+dataId+"1-poses_xyz.txt")
print(vo_calc.shape)
plt.plot(vo_calc[:,2],vo_calc[:,0])

### GPS/INS Ground Truth for trajectories
#### Interpolation method to align vo and ins data, and obtain ins coordinates for images

In [ ]:
def getImageCords(insData,voData):
    # interpolate latitude and longitude with respect to the timestamp index of these values
    insDataPart = insData[:,[0,2,3]].astype(float)
    latFunc = interpolate.interp1d(insDataPart[:,0],insDataPart[:,1])
    longFunc = interpolate.interp1d(insDataPart[:,0],insDataPart[:,2])

    # if VO data starts before GPS, we will have to skip it
    numSkipVO = np.min(np.argwhere((voData[:,1] - insDataPart[0,0]) > 0))
    print('Skipped',numSkipVO," VO values to align VO with GPS data")
    # Get lat and long for given timestamps in vo
    lats = latFunc(voData[numSkipVO:,1])
    longs = longFunc(voData[numSkipVO:,1])

    # Plot the cords
    plt.plot(longs,lats)
    plt.title("Image Co-ordinates")
    plt.show()

    imageCords = np.vstack((np.arange(longs.shape[0])+numSkipVO,lats,longs)).transpose()

    return imageCords

In [ ]:
# Load GPS/INS data
insFile = folderPath+dataId+"gps/ins.csv"
insData = np.loadtxt(insFile,skiprows=1,
                    delimiter=',',dtype='str')
print(insData.shape)
plt.plot(insData[:,3],insData[:,2])
plt.title("INS Ground Truth Data")
plt.show()

imageCords = getImageCords(insData,voData)

# Save the cords
# np.savetxt(folderPath+dataId+"imageCords.txt",imageCords,fmt=['%d','%f','%f'])

### Speed-Normalized / Constant-Distance Timestamps 
Following code is useful for sampling images at constant distance if their corresponding odometry/gps/ins is given

In [ ]:
def getUsefulInds(deltaR_n_norm,unit=1):
    # Store only the indices that are given units apart
    usefulInds = []
    # Keep a record of usefulInds wrt each raw index
    usefulIndsCounter = []
    cumSum = 0
    for i in range(len(deltaR_n_norm)):
        cumSum += deltaR_n_norm[i]
        while cumSum > unit:
            usefulInds.append(i)
            cumSum -= unit
        usefulIndsCounter.append(len(usefulInds))
    return np.array(usefulInds), np.array(usefulIndsCounter)
       
    
# Odomdata asssumed to have 3 cols - timestamp, x, y  
# newTS assumed to have 1 col - timestamp
def getOdomForTS(odomData,newTS):
    xFunc = interpolate.interp1d(odomData[:,0],odomData[:,1])
    yFunc = interpolate.interp1d(odomData[:,0],odomData[:,2])
    
    # if newTS data starts before odomTS, we will have to skip it
    numSkipTS = np.min(np.argwhere((newTS - odomData[0,0]) > 0))
    lastTSIdx = newTS.shape[0]
    if newTS[-1] - odomData[-1,0] > 0:
        lastTSIdx = np.min(np.argwhere((newTS-odomData[-1,0])>0))
    print('Skipped',numSkipTS," values to align newTS with odomTS")
    # Get x and y for new timestamps
    xNew = xFunc(newTS[numSkipTS:lastTSIdx])
    yNew = yFunc(newTS[numSkipTS:lastTSIdx])
    
    # Plot the cords
    plt.plot(yNew,xNew)
    plt.title("Image Co-ordinates")
    plt.show()
    
    newTsCords = np.vstack((np.arange(xNew.shape[0])+numSkipTS,xNew,yNew)).transpose()

    return newTsCords

def sampleTSConstantDistance(imgCords1,factor=2):   
    # Calc displacement
    disp1 = np.linalg.norm(imgCords1[1:,1:] - imgCords1[:-1,1:],axis=1)
    # append a disp value of zero at the beginning
    disp1 = np.insert(disp1,0,0,)
    print("Total image frames - ",disp1.shape)

    # Calc avg distance travelled per image
    avgDispPerImg1 = np.mean(disp1)
    print("Avg Displacement per Image ", avgDispPerImg1)

    # Get speed normalized / constant distance indices
    usefulInds1, usefulIndsCounter1 = getUsefulInds(disp1,avgDispPerImg1*factor)
    assert(len(usefulIndsCounter1)==len(disp1))
    print("Constant-Distance Sampled Image frames - ",usefulInds1.shape)

    # Plot the new co-ordinates separated by constant distance
    plt.plot(imgCords1[usefulInds1,2],imgCords1[usefulInds1,1])
    plt.title("Image Co-ordinates separated by constant distance\n")
    plt.show()

    # Plot the speed
    plt.plot(usefulIndsCounter1)
    plt.title("Speed Profile")
    plt.show()
    
    return usefulInds1, usefulIndsCounter1

#### Following code calls functions above and requires prior loading of only ins/gps/odom data  and folderPath+dataID for input 

In [ ]:
sensor = "mono_rear"
newTimeStamps = np.loadtxt(folderPath+dataId+sensor+".timestamps")[:,0]

# Obtain the co-ordinates for new sensor
imgCords = getOdomForTS(insData[:,[0,2,3]].astype(float),newTimeStamps)

# Obtain the sampled indices
sampledInds, sampledIndsIndex = sampleTSConstantDistance(imgCords,2)

# print("Writing to path - ", folderPath+dataId)
# np.savetxt(folderPath+dataId+sensor+"_sampled_indices.txt",sampledInds,fmt='%d')
# np.savetxt(folderPath+dataId+sensor+"_sampled_indices_index.txt",sampledIndsIndex,fmt='%d')

##  ---- TO DO ---- 

### Place Recognition Ground Truth 

In [ ]:
imageCords1 = np.loadtxt(folderPath+dataId+"imageCords.txt")
imageCords2 = np.loadtxt(folderPath+dataId2+"imageCords.txt")

In [ ]:
# if 2 is query, then matching index (of ref) for its each image is stored
gtPR21 = []
searchRegion = 2000
diffVecs = []
for i in range(imageCords2.shape[0]):
    cord2 = imageCords2[i,1:]
    searchStart = 0#min(max(0,i-searchRegion),imageCords1.shape[0]-searchRegion)
    searchEnd = imageCords1.shape[0]#min(imageCords1.shape[0],i+searchRegion)
    diffVec = (np.sum(abs(np.subtract(imageCords1[searchStart:searchEnd,1:],cord2)),axis=1))
    matchIdx = np.argmin(diffVec)
    diffVecs.append(diffVec)
    gtPR21.append(searchStart+matchIdx)
#     for j in range(imageCords1.shape[0]):
#         cord1 = imageCords1[j,1:]
        
#         if 

In [ ]:
plt.plot(gtPR21)

In [ ]:
plt.imshow(np.array(diffVecs))
# print(len(diffVecs[2220]))